In [20]:
import nibabel as nib
import os
import numpy as np
import pandas as pd
from glob import glob
import re
import multiprocess as mp
import glob
import time
%matplotlib inline

In [3]:
datadir = os.path.abspath('../20190801')

# Help functions

In [4]:
def fslstats_vol(f):
    import subprocess
    output = subprocess.check_output(['fslstats', f, '-V']).decode('utf-8').split( ' ')
    return float(output[0]), float(output[1])

def fslstats_mean(f):
    import subprocess
    output = subprocess.check_output(['fslstats', f, '-M']).decode('utf-8')
    return float(output)

Volume = Voxel count * Probability, the voxel belong to the Lesion (intensity in probability map)

In [24]:
def calc_vols(i):
    try:
        lpa = os.path.join(datadir, '{0}/flair/ples_lpa_m{0}.flair_ro_n4_2t1Warped.nii'.format(i))
        lga = os.path.join(datadir, '{0}/ples_lga_0.3_rm{0}.flair_ro.nii'.format(i))
        return {i:{'lpa': fslstats_vol(lpa)[0] * fslstats_mean(lpa), 
                'lga': fslstats_vol(lga)[0] * fslstats_mean(lga)}}
    except:
        print('Problem with {}'.format(i))
        
def MPDicts2dict(list_of_dicts):
    """
    Convert a list of dictionaries from multiprocess to dictionary
    """
    d = dict()
    for i in list_of_dicts:
        if i.get():
            d.update(i.get())
    return d

## Find subject IDs

In [18]:
sub_ids = [ i for i in os.listdir(datadir) if i.startswith('SMART')]

In [19]:
i = sub_ids[0]
lpa = os.path.join(datadir, '{0}/flair/ples_lpa_m{0}.flair_ro_n4_2t1Warped.nii'.format(i))
lga = os.path.join(datadir, '{0}/ples_lga_0.3_rm{0}.flair_ro.nii'.format(i))


## Get Volumes

In [21]:
pool = mp.Pool(processes=8)
results = [pool.apply_async(calc_vols, args=(i,)) for i in sub_ids]

Problem with SMART021
Problem with SMART040_T2_2
Problem with SMART_REF019


In [25]:
output = MPDicts2dict(results)

In [28]:
df = pd.DataFrame.from_dict(output, orient = 'index')

In [29]:
df.to_csv('vols.csv')